In [2]:

library(readxl)
library(readxlsb)
library(data.table)
library(rpart)
library(rpart.plot)
library(readxl)
library(stringr)
library(pmml)
library(partykit)

library(ranger)
library(xgboost)
library(ggplot2)
library(ROCR)
library(SHAPforxgboost)

Loading required package: XML

Loading required package: grid

Loading required package: libcoin

Loading required package: mvtnorm



In [3]:
dt <- readRDS('../data/DB_perimetro.RDS')

In [105]:
col_colore <- colnames(dt)[grepl('colore -',colnames(dt) )]

In [106]:
col_esito <- colnames(dt)[grepl('esito -',colnames(dt) )]


In [107]:
col_target <- c('rating_fine_periodo_adj','rating_fine_periodo_ricondotto',
'rating_periodo_adj','rating_fine_periodo_adj','mean_of_rating_periodo_adj')

In [108]:
c1<- c('stato_pratica','chiave_db','abi','abi_pef5','filiale_pef5','anno_pef5','progressivo_pef5',
       'provenienza_pratica','descrizione_provenienza','data_delibera_o_annull_fido',
       'annomese_delibera_o_annull_fido','condice_consociata','user_istruttoria','data_istruttoria',
       'user_delibera','sndg_ceduto_old')

In [109]:
c2<-c('macrosegmento_attuale','rating_attuale_ricondotto','cod_mod_bws_attuale',
      'prima_data_attrib_primula','primo_segmento_primula','primo_modello_primula','primo_rating_primula',
      'monopratica_o_monocolore','accordato_medio_pro_soluto','utilizzato_medio_pro_soluto',
      'accordato_medio_prosolvendo','utilizzato_medio_prosolvendo','accordato_medio_altri_crediti',
      'acc_operativo_medio_altri_credit','utilizzato_medio_altri_crediti','saldo_creditori_medio_altri_cred',
      'accordato_medio_altro_fac','acc_oper_medio_altro_fact','utilizzato_medio_altro_fac',
      'saldicredit_medio_altro_fac','data_richiesta','ragio_sociale_ceduto','ragio_sociale_cedente',
      'id_cedente...34','conteggio_giallo_esito_finale_giallo','conteggio_rosso',
      'appoggio - esito_final_giallo e monocontrollo_giallo',
      'appoggio - esito_final_rosso e monocontrollo_rosso','esito_final_giallo e monocontrollo_giallo',
      'esito_final_rosso e monocontrollo_rosso','flag_utilizzato=0','perimetro analisi descrittive',
      'perimetro_finale_analisi td','perimetro_no_default_t0','perimetro_si_default_tfinal',
      'cluster_operazione_concessa_applica','rating_appoggio','rating_appoggio2','rating',
      'sag_appoggio','sag_appoggio2','sag','id_cedente_appoggio','id_cedente...228',
      'chiave_limiti_soc','limite_max','cluster_finale_operazione_concessa_applica',
      'sogg_estero_risk','macrosegmento_attuale lc','flag_univocita_cont_giallo',
      'cluster_numerosità_contr_gialli','cluster_numerosità_contr_rossi',
      'flag_univoco/compresenza_gialli','flag_imp_fido_orig_<limite','richiesto_vs_limite_v2',
      'perimetro_standard','scarto_vs_limite','cluster_scarto','importo origi+appl concessa',
      'cluster_rating_fascia','m3-m4','m1-m2','i5-i6','i4','i3','i1-i2','nuovi_verdi',
      'nuovo_esito_finale','nuovi_verdi_senza_soglia','nuovo_esito_senza soglia')

In [110]:
c3 <- c('importo concesso applicazione','divisa concessa applicazione','tipo operazione concessa applica',
        'quota prosoluto - importo','quota prosoluto - ft gf','quota prosoluto - ft pcr',
        'quota prosolvendo - importo','quota prosolvendo - ft gf','quota prosolvendo - ft pcr',
        'tipo operazione fido','importo fido','divisa fido','filiale_factoring_ceduto','filiale_cedente',
        'settore_cedente','descrizione_settore','rae_cedente','rae_descr_cedente','sae_cedente',
        'sae_descr_cedente','numero_tot_fidi','count_fidi_in_conferma','sum_import_fidi_concessione',
        'sum_import_fidi_aumento','data_richiesta_fido','isr_data','isr_codice','isr_descrizione',
        'data_perfez_contratto','semaforo_primo_livello','prefisso_mod_rating_ceduto',
        'modello_rating_ceduto','rating_ceduto','cib_bdt_cedente','prefisso_mod_rating_cedente',
        'modello_rating_cedente','rating_cedente','sag_ceduto','sag_cedente','flag_ubi_factor')

In [111]:
c4 <- c('pratica_testo','flag_monofido','presenza_fido_580_590','presenza_fido_580_590_c',
        'congruenza_prodotto_fido','flag_applicazione_univoca','flag_finale','istruttore','deliberante',
        'sndg_ceduto_new','flag_ditta_individuale','sndg_ceduto','tipo_valutazione_del_ceduto',
        'flag_valutazione_vpc_sul_ceduto','flag_presenza_sndg_ceduto','presenza_rating','rating_inizio_periodo',
        'periodo_osservazione_in_corso','dat_attr_rat_per_compreso','dat_attr_rat_t0','dat_attr_rat_per_finale',
        'data_attr_rat_fine_per_0101160','data_attr_rat_fin_per_31122999','prima_data_attr_primula_per_ante',
        'media_rating_per_ante','prima_data_attrib_prim_per_post','cod_segm_per_compreso','cod_segm_t0',
        'cod_mod_bws_per_compreso','cod_mod_bws_t0','cod_classe_cra_per_compreso','cod_classe_cra_t0',
        'cod_colore_classe_per_compreso','anno_delibera','mese_delibera','delta_fido_selezionato',
        'deltafido580590_inv_prod','deltafido_capofila_non_580590')

In [112]:
cols <- setdiff(colnames(dt),c(col_colore,col_esito, c1,c2,c3,c4,col_target))
names(cols)<-gsub("input - ","",setdiff(colnames(dt),c(col_colore,col_esito, c1,c2,c3,c4,col_target)))
names(cols)<- gsub("\\s+","_",gsub('[^a-zA-Z0-9]', " ",names(cols)))

In [314]:
dt_clean <- copy(dt[,..cols])
colnames(dt_clean)<- names(cols)

In [315]:
## elimino righe non in euro, sono poche (2) quindi non influiscono, altrimenti avrei dovuto fare la conversione degli importi per poterli confrontare
dt_clean <- dt_clean[divisa_richiesta_applicazione =='EURO',]

In [316]:
dt_clean[,limiti_societari := gsub("\\."," ",limiti_societari)]
dt_clean[,limiti_societari := gsub(":"," ",limiti_societari)]
dt_clean[,limiti_societari := gsub("\\s+"," ",limiti_societari)]

dt_clean[,ced := gsub(".*CED (.+) SAG.*", "\\1",limiti_societari )]
dt_clean[,sag := gsub(".*SAG (.+) RTNG.*", "\\1",limiti_societari )]
dt_clean[,rat := gsub(".*RTNG (.+) ESC.*", "\\1",limiti_societari )]
dt_clean[,esc := gsub(".*ESC (.+) RIS.*", "\\1",limiti_societari )]
dt_clean[,aput := ifelse(grepl("APPL",limiti_societari),
                         gsub(".*APUT (.+) APPL.*", "\\1",limiti_societari ),
                         gsub(".*APUT ", "\\1",limiti_societari ))]
dt_clean[,aput := as.numeric(gsub(",",".",aput))]

dt_clean[,ris := gsub(".*RIS (.+) APUT.*", "\\1",limiti_societari )]
dt_clean[,ris := as.numeric(gsub(",",".",ris))]

In [317]:
dt_clean[,caus_caus_neg := gsub(".*CAUS:(.+)STATO:.*", "\\1",fido_con_causale_negativ )]
dt_clean[,stato_caus_neg := gsub(".*STATO:(.+)FIDO.*", "\\1",fido_con_causale_negativ )]
dt_clean[,tipo_caus_neg := gsub(".*TIPO:", "\\1",fido_con_causale_negativ )]

In [318]:
dt_clean[,limiti_societari_as_is := gsub("\\."," ",limiti_societari_as_is)]
dt_clean[,limiti_societari_as_is := gsub(":"," ",limiti_societari_as_is)]
dt_clean[,limiti_societari_as_is := gsub("\\s+"," ",limiti_societari_as_is)]

dt_clean[,ced_as_is := gsub(".*CED (.+) SAG.*", "\\1",limiti_societari_as_is )]
dt_clean[,sag_as_is := gsub(".*SAG (.+) RTNG.*", "\\1",limiti_societari_as_is )]
dt_clean[,rat_as_is := gsub(".*RTNG (.+) ESC.*", "\\1",limiti_societari_as_is )]
dt_clean[,escl_as_is := gsub(".*ESCL", "\\1",limiti_societari_as_is )]

In [319]:
dt_clean[,imprese_collegate_proce := gsub(":"," ",imprese_collegate_proce)]
dt_clean[,imprese_collegate_proce := gsub("\\s+"," ",imprese_collegate_proce)]

dt_clean[,sensore_proc := gsub(".*PROC(.+)STATO.*", "\\1",imprese_collegate_proce )]
dt_clean[,stato_soc := gsub(".*SOC ", "\\1",imprese_collegate_proce )]


In [320]:
dt_clean[,approvato := gsub("\\s+"," ",approvato)]

dt_clean[,approvato_dettaglio := gsub(".*APPROVATO: (.+) APPL.*", "\\1",approvato )]
dt_clean[,appl_ric := gsub(".*RIC: (.+) DA.*", "\\1",approvato )]
dt_clean[,da_sanare := gsub(".*DA SANARE", "\\1",approvato )]
dt_clean[,approvato_dettaglio := as.numeric(gsub(",",".",approvato_dettaglio))]
dt_clean[,appl_ric := as.numeric(gsub(",",".",appl_ric))]

In [321]:
dt_clean[,cr_presenza_sofferenze := gsub("\\s+"," ",cr_presenza_sofferenze)]

dt_clean[,cr_ps_acc := gsub(".*ACC: (.+) UTI:.*", "\\1",cr_presenza_sofferenze )]
dt_clean[,cr_ps_acc := as.numeric(gsub(",",".",cr_ps_acc))]

dt_clean[,cr_ps_uti := gsub(".*UTI: (.+) SCO:.*", "\\1",cr_presenza_sofferenze )]
dt_clean[,cr_ps_uti := as.numeric(gsub(",",".",cr_ps_uti))]

dt_clean[,cr_ps_sconf := gsub(".*SCO:", "\\1",cr_presenza_sofferenze )]
dt_clean[,cr_ps_sconf := as.numeric(gsub(",",".",cr_ps_sconf))]

In [322]:
dt_clean[,cr_utilizzato_accorda := gsub("\\s+"," ",cr_utilizzato_accorda)]

dt_clean[,cr_ua_acc := gsub(".*ACC: (.+) TOT UTI:.*", "\\1",cr_utilizzato_accorda )]
dt_clean[,cr_ua_acc := as.numeric(gsub(",",".",cr_ua_acc))]

dt_clean[,cr_ua_uti := gsub(".*TOT UTI:", "\\1",cr_utilizzato_accorda )]
dt_clean[,cr_ua_uti := as.numeric(gsub(",",".",cr_ua_uti))]



In [323]:
dt_clean[,scaduto_20_outstanding := gsub("\\s+"," ",scaduto_20_outstanding)]

dt_clean[,scaduto_out_scad := gsub(".*SCAD: (.+) TOT OUTS:.*", "\\1",scaduto_20_outstanding )]
dt_clean[,scaduto_out_scad := as.numeric(gsub(",",".",scaduto_out_scad))]

dt_clean[,scaduto_out_outs := gsub(".*TOT OUTS:", "\\1",scaduto_20_outstanding )]
dt_clean[,scaduto_out_outs := as.numeric(gsub(",",".",scaduto_out_outs))]



In [324]:
dt_clean[,rat_cluster := rat]
dt_clean[grepl('R',rat_cluster),rat_cluster := "R"]
dt_clean[grepl('M',rat_cluster),rat_cluster := "M"]
dt_clean[grepl('I',rat_cluster),rat_cluster := "I"]

In [325]:
dt_clean[,soggetto_cedente := gsub("[^0-9]","",soggetto_cedente)]
dt_clean[,presenza_di_fidi_cedente := gsub("[^0-9]","",presenza_di_fidi_cedente)]
dt_clean[,parte_correlata := gsub("[^0-9]","",parte_correlata)]

In [326]:
### controllo RIS in limiti societari, si parla con altre colonne di importo? NO

# dt_clean[, c(col_master[grepl('imp',col_master)],'ris', 'limiti_societari'), with = FALSE]

In [327]:
## colonne che hanno un solo valore
col_1_val <- lapply(colnames(dt_clean),function(x){
    if(length(unique(unlist(dt_clean[, eval(x), with = FALSE])))<2){
        return(x)
    }else return(NULL)
})

col_1_val <- unlist(col_1_val)


In [328]:
col_anagrafica <- c('num_pratica','progress_pratica','tipo_pratica','codice_fiscale_ceduto','ndg_ceduto',
                    'andi_ndg_super','ndg_cedente','sndg_cedente','codice_fiscale_cedente','id_cedente',
                    'ndg_settoriale_mancante','codice_fiscale_p_iva_no','codice_fiscale_gia_pres',
                    'codice_fiscale_gia_0001','codice_fiscale_formalmen','confirming_codice_fisc')
col_extra_to_rm <- c('importo_richiesto_ad','incongruenza_tra_localit','rae_descr_ceduto','sae_descr_ceduto','divisa_richiesta_applicazione',
                     'rating_scaduto','fido_con_causale_negativ','data_stile_numerico','approvato','applicazione_con_causale','bilanci_passivo_corrent','stato_cedente')
col_split <- c('limiti_societari','limiti_societari_as_is','scaduto_20_outstanding','cr_utilizzato_accorda','cr_presenza_sofferenze','imprese_collegate_proce')
## applicazione_con_causale e bilanci_passivo_corrent vanno analizzate

In [329]:
col_master <- setdiff(colnames(dt_clean),c(col_anagrafica,col_extra_to_rm,col_1_val,col_split))

In [330]:
col_master[grepl('imp',col_master)]

[1] "importo_richiesto_applicazione" "importo_richiesto_a_zero"      
[3] "importo_richiesto"              "importo_originale"             
[5] "importo_originario"             "import_origin_appl_prosoluto"  
[7] "import_origin_appl_prosolvendo" "importo_fido_adj"

In [331]:
master <- dt_clean[,..col_master]
master<- master[,which(unlist(lapply(master, function(x)!all(is.na(x))))),with=F]
master<- master[,which(unlist(lapply(master, function(x)!all(is.na(x))))),with=F]
master$target <- factor(master$target, levels = c('OK',"DEFAULT"))

In [332]:

master <- cbind.data.frame(lapply(master, function(x) {
  if (all(unique(x) %in% c(0,1)))
    as.factor(c("NO", "YES")[x+1])
  else if (all(is.character(x)))
    as.factor(x)
  else
    x
}))
setDT(master)

In [421]:
set.seed(1704)
rows <- sample(1:nrow(master), 100000, replace=F)
modello2 <- rpart(target ~. ,
                data=master[rows,],
                method="class", control=rpart.control(cp = 1e-2))#metodo class o anova 

In [422]:
modello2

n= 100000 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 100000 3098 OK (0.969020000 0.030980000)  
   2) rating_attuale=I1,I1.B,I1.C,I1.D,I1.E,I2,I3,I4,I5,I6,M1,M2,M3,M4,R1,R2,R3,R4,RT01,RT02,RT03,RT04,RT05,RT06,RT07,RT08,RT09,RT10,RT11,RT12,RT13,RT14 92535  685 OK (0.992597396 0.007402604) *
   3) rating_attuale=D,R5,UR 7465 2413 OK (0.676758205 0.323241795)  
     6) rating_medio_post_stipula>=0.3333333 6402 1682 OK (0.737269603 0.262730397)  
      12) rating_attuale=R5,UR 3337  506 OK (0.848366797 0.151633203)  
        24) rating_medio_post_stipula< 15.4881 2000  139 OK (0.930500000 0.069500000) *
        25) rating_medio_post_stipula>=15.4881 1337  367 OK (0.725504862 0.274495138)  
          50) rating_medio_post_stipula>=15.94868 1001  137 OK (0.863136863 0.136863137) *
          51) rating_medio_post_stipula< 15.94868 336  106 DEFAULT (0.315476190 0.684523810) *
      13) rating_attuale=D 3065 1176 OK (0.616313214 0.383686786)  
        26) ced

In [423]:
options(repr.plot.width=8, repr.plot.height=7)
pdf('prova2.pdf')
rpart.plot(modello2, type=2, fallen.leaves=F, yesno=2, extra = 107,box.palette="-RdGn")
dev.off()

pdf 
  2

In [424]:
pmod2 <- pmml(modello2, model_name = "VPC Tree Model",
app_name = NULL,
description = NULL,
copyright = NULL)

In [425]:
saveXML(pmod2, file="tree_v2.xml")

[1] "tree_v2.xml"

In [426]:
pred_mod2<- predict(modello2,master[-rows,])
pred_mod2<- as.data.table(pred_mod2)

dt_appo_pred <- master[-rows,] 
dt_appo_pred$pred_vpc <- pred_mod2$DEFAULT

table(dt_appo_pred[,c('pred_vpc','target')])

                     target
pred_vpc                 OK DEFAULT
  0.00740260441994921 33111     237
  0.0147058823529412     19       1
  0.0685714285714286     53       3
  0.0695                683      54
  0.136863136863137     335      54
  0.299465240641711     553     228
  0.555555555555556     145     169
  0.68452380952381       38      86
  0.875609756097561      48     271

In [427]:
table(pred_mod2[,2]>0.25,master[-rows,target] )

       
           OK DEFAULT
  FALSE 34201     349
  TRUE    784     754

In [428]:
nrow(master[-rows,][target == 'DEFAULT',])

[1] 1103

In [429]:
table(dt_appo_pred[pred_vpc <0.1 & target == "DEFAULT",rat_cluster])


  D   I   M   R 
  2  12  27 246 

In [389]:
colnames(master)

[1] "settore_ceduto"                   "divisione_ceduto"                
 [3] "tipo_applicazione"                "importo_richiesto_applicazione"  
 [5] "tipo_operazione_richiesta_applic" "ft_factoring"                    
 [7] "tipologia_ft_factoring_continua"  "indicatore_se_fido_capofila_o_pr"
 [9] "descr_stato_cedente"              "esito_finale_vpc"                
[11] "cedente_revocato"                 "cedente_travel"                  
[13] "soggetto_estero"                  "tipo_operazione_riduzion"        
[15] "importo_richiesto_a_zero"         "soggetto_eepp"                   
[17] "stato_sab"                        "parte_correlata"                 
[19] "presenza_di_fidi_cedente"         "soggetto_cedente"                
[21] "rapporti_in_osservazione"         "stato_societario"                
[23] "protesti_certi"                   "protesti_dubbi"                  
[25] "fallimenti_certi"                 "pregiudizievoli_gravi"           
[27] "ateco_bkit_obbligatorio"          "timeout_scaduto1"                
[29] "nessuna_posizione_incroc"         "inserimento_non_consenti"        
[31] "timeout_scaduto2"                 "timeout_scaduto3"                
[33] "errore_generico_risotto"          "importo_richiesto"               
[35] "right_num_pratica"                "right_progress_pratica"          
[37] "rae_ceduto"                       "sae_ceduto"                      
[39] "importo_originale"                "polizza_diretta_39_"             
[41] "polizza_indiretta_41_"            "tipologia_ft_factoring_continu2" 
[43] "importo_originario"               "import_origin_appl_prosoluto"    
[45] "import_origin_appl_prosolvendo"   "rating_medio_post_stipula"       
[47] "cod_colore_classe_t0"             "rating_attuale"                  
[49] "cod_prov_attuale"                 "importo_fido_adj"                
[51] "numerofidi_pratica"               "delta_applicazione"              
[53] "tipo_operazione_fido_aggregato"   "tipo_operaz_concessa_appl_aggreg"
[55] "rating_iniz_periodo_ricondotto"   "macrosegmentot0"                 
[57] "macrosegmento_per_compreso"       "target"                          
[59] "ced"                              "sag"                             
[61] "rat"                              "esc"                             
[63] "aput"                             "ris"                             
[65] "caus_caus_neg"                    "stato_caus_neg"                  
[67] "tipo_caus_neg"                    "ced_as_is"                       
[69] "sag_as_is"                        "rat_as_is"                       
[71] "escl_as_is"                       "sensore_proc"                    
[73] "stato_soc"                        "approvato_dettaglio"             
[75] "appl_ric"                         "da_sanare"                       
[77] "cr_ps_acc"                        "cr_ps_uti"                       
[79] "cr_ua_acc"                        "cr_ua_uti"                       
[81] "scaduto_out_scad"                 "scaduto_out_outs"                
[83] "rat_cluster"

In [310]:
unique(gsub(".*PROC(.+)STATO.*", "\\1",dt_clean$imprese_collegate_proce ))

[1] NA     " "    " PP " " PC " " ER " " ND "

In [313]:
dt_clean

num_pratica,progress_pratica,tipo_pratica,codice_fiscale_ceduto,ndg_ceduto,andi_ndg_super,settore_ceduto,ndg_cedente,divisione_ceduto,tipo_applicazione,⋯,appl_ric,da_sanare,cr_ps_acc,cr_ps_uti,cr_ps_sconf,cr_ua_acc,cr_ua_uti,scaduto_out_scad,scaduto_out_outs,rat_cluster
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
102230026,1,3,9054770962,4608870000000,NA,18,6182450000000,BDT,PLAFOND,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,R
102230029,2,3,2695820304,4495180000000,NA,6,3288980000000,BDT,PLAFOND,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,R
102230030,1,2,80002210104,2012160000000,NA,89,3288980000000,BDT,PLAFOND,⋯,0,,NA,NA,NA,NA,NA,NA,NA,NA
102230031,1,3,3503350260,3340210000000,NA,6,3288980000000,BDT,PLAFOND,⋯,0,,NA,NA,NA,NA,NA,NA,NA,I
102230032,1,3,3679130264,203665000000,NA,6,3288980000000,BDT,PLAFOND,⋯,0,,NA,NA,NA,NA,NA,NA,NA,I
102230034,1,2,3917900874,202162000000,NA,6,3288980000000,BDT,PLAFOND,⋯,0,,NA,NA,NA,NA,NA,NA,NA,NA
102230035,1,3,3930791219,3225830000000,NA,6,3288980000000,BDT,PLAFOND,⋯,0,,NA,NA,NA,NA,NA,NA,NA,R
103230023,1,3,1924290248,1441010000000,NA,6,1483160000000,BDT,PLAFOND,⋯,0,,NA,NA,NA,NA,NA,NA,NA,I
103230027,1,3,3919160659,7651580000000,NA,89,3980230000000,BDT,PLAFOND,⋯,0,,NA,NA,NA,NA,NA,NA,NA,R


In [311]:
unique(dt_clean$ced_as_is)

[1] "398188884" NA

In [185]:
unique(dt_clean$stato_cedente)

[1] "2" "1" "3" "6" "7" "5" "8"

In [184]:
colnames(dt)[grepl('ced',colnames(dt))]

[1] "sndg_ceduto_old"                  "codice_fiscale_ceduto"           
 [3] "ndg_ceduto"                       "filiale_factoring_ceduto"        
 [5] "settore_ceduto"                   "ndg_cedente"                     
 [7] "divisione_ceduto"                 "sndg_cedente"                    
 [9] "codice_fiscale_cedente"           "id_cedente"                      
[11] "stato_cedente"                    "cedente_si/no"                   
[13] "descr_stato_cedente"              "filiale_cedente"                 
[15] "settore_cedente"                  "colore - cedente revocato"       
[17] "colore - cedente travel"          "colore - export facile-sia ceden"
[19] "colore - presenza di fidi cedent" "colore - soggetto cedente"       
[21] "colore - procedure concorsuali"   "colore - cedente appartiene a gr"
[23] "esito - cedente revocato"         "esito - cedente travel"          
[25] "esito - export facile-sia cedent" "esito - presenza di fidi cedente"
[27] "esito - soggetto cedente"         "esito - procedure concorsuali"   
[29] "esito - cedente appartiene a gru" "input - cedente revocato"        
[31] "input - cedente travel"           "input - export facile-sia cedent"
[33] "input - presenza di fidi cedente" "input - soggetto cedente"        
[35] "input - procedure concorsuali"    "input - cedente appartiene a gru"
[37] "rae_ceduto"                       "rae_descr_ceduto"                
[39] "sae_ceduto"                       "sae_descr_ceduto"                
[41] "rae_cedente"                      "rae_descr_cedente"               
[43] "sae_cedente"                      "sae_descr_cedente"               
[45] "prefisso_mod_rating_ceduto"       "modello_rating_ceduto"           
[47] "rating_ceduto"                    "cib_bdt_cedente"                 
[49] "prefisso_mod_rating_cedente"      "modello_rating_cedente"          
[51] "rating_cedente"                   "sag_ceduto"                      
[53] "sag_cedente"                      "sndg_ceduto_new"                 
[55] "sndg_ceduto"                      "tipo_valutazione_del_ceduto"     
[57] "flag_valutazione_vpc_sul_ceduto"  "flag_presenza_sndg_ceduto"       
[59] "ragio_sociale_ceduto"             "ragio_sociale_cedente"           
[61] "id_cedente...34"                  "id_cedente_appoggio"             
[63] "id_cedente...228"

In [103]:
dt[3,cols['limiti_societari'], with = FALSE]

input - limiti societari
<chr>
NA


In [ ]:
master <- dt_clean[,..col_master]
master<- master[,which(unlist(lapply(master, function(x)!all(is.na(x))))),with=F]
master<- master[,which(unlist(lapply(master, function(x)!all(is.na(x))))),with=F]